In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lor').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [5]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
#create the modelobject , keep everything as default since our dataframe has labels and features alreday
log_reg = LogisticRegression()

In [11]:
fitted_log_reg = log_reg.fit(data)

In [13]:
log_summary = fitted_log_reg.summary

In [16]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



/home/ahmad/Desktop/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [21]:
log_summary.predictions.select(['label','prediction']).show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [22]:
train_data,test_data = data.randomSplit([0.7,0.3])

In [23]:
final_model = LogisticRegression()

In [24]:
fitted_final = final_model.fit(train_data)

In [26]:
evalaution = fitted_final.evaluate(test_data)

In [27]:
evalaution_df = evalaution.predictions

/home/ahmad/Desktop/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [28]:
type(evalaution_df)

pyspark.sql.dataframe.DataFrame

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [30]:
my_eval = BinaryClassificationEvaluator()

In [31]:
my_final_roc = my_eval.evaluate(evalaution_df)

In [32]:
type(my_final_roc)

float

In [34]:
# thats a perfect fit
my_final_roc

1.0